
- logarithm
- feat_eng
- target_cut

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import warnings

import funcs

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import (
    ExtraTreesRegressor,
    RandomForestRegressor,
    VotingRegressor,
)
from sklearn.impute import KNNImputer
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import MaxAbsScaler
from xgboost import XGBRegressor

%matplotlib inline
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
# df_train = df_train.drop(['Id'], axis=1)
# df_test = df_test.drop(['Id'], axis=1)
df_train.rename(
    columns={
        "Strength": "target",
        "Superplasticizer": "SP",
        "Fly Ash": "FA",
        "Water": "W",
        "Coarse Aggregate": "Ag",
        "Fine Aggregate": "Af",
        "Blast Furnace Slag": "BFS",
        "Cement": "C",
        "Age": "t",
    },
    inplace=True,
)
df_train.info()
# df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      800 non-null    int64  
 1   C       800 non-null    float64
 2   BFS     800 non-null    float64
 3   FA      800 non-null    float64
 4   W       800 non-null    float64
 5   SP      800 non-null    float64
 6   Ag      800 non-null    float64
 7   Af      800 non-null    float64
 8   t       800 non-null    int64  
 9   target  800 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 62.6 KB


- Id - идентификатор состава.
- CementComponent - количество цемента в смеси.
- BlastFurnaceSlag - количество доменного шлака в смеси.
- FlyAshComponent - количество зольной пыли в смеси.
- WaterComponent - количество воды в смеси.
- SuperplasticizerComponent - количество суперпластификатора в смеси.
- CoarseAggregateComponent - количество заполнителя с грубой фракцией в смеси.
- FineAggregateComponent - количество заполнителя с мелкой фракцией в смеси.
- Age - время высыхания в днях.
- Strength - прочность получившегося бетона (Целевая переменная)


In [5]:
sorted(df_train.t.unique())

[1, 3, 7, 14, 28, 56, 90, 91, 100, 120, 180, 270, 360, 365]

In [6]:
df_train.loc[df_train.t == 360, "t"] = 365
df_train.loc[df_train.t == 91, "t"] = 90
df_train.loc[df_train.t.isin([100, 120]), "t"] = 120
df_train.loc[df_train.t == 1, "t"] = 3
# df_train.loc[df_train.t.isin([270, 365]), "t"] = 180
print(sorted(df_train.t.unique()))
print(df_train.t.value_counts())

[3, 7, 14, 28, 56, 90, 120, 180, 270, 365]
t
28     321
7      102
3      100
56      73
90      67
14      49
120     42
180     19
365     16
270     11
Name: count, dtype: int64


In [7]:
df_train.columns

Index(['Id', 'C', 'BFS', 'FA', 'W', 'SP', 'Ag', 'Af', 't', 'target'], dtype='object')

In [8]:
duplicates = df_train[df_train.drop(columns=["target", "t", "Id"]).duplicated()]
len(duplicates)

437

In [9]:
# Предположим, что данные содержат следующие столбцы:
# 'Id', 'CementComponent', 'BlastFurnaceSlag', 'FlyAshComponent',
# 'WaterComponent', 'SuperplasticizerComponent', 'CoarseAggregateComponent',
# 'FineAggregateComponent', 'Age', 'Strength'

# Убираем дублирующиеся строки состава смеси, кроме 'Age' и 'Strength'
data_grouped = df_train.groupby(
    ["C", "BFS", "FA", "W", "SP", "Ag", "Af"], as_index=False
)

# Создаем новый "выпрямленный" DataFrame
df_flat = data_grouped.apply(
    lambda group: pd.Series(
        {
            age: (
                group[group["t"] == age]["target"].values[0]
                if age in group["t"].values
                else None
            )
            for age in sorted(df_train["t"].unique())
        }
    )
).reset_index()

# Добавляем названия для столбцов
df_flat.columns = list(df_flat.columns[: -len(df_train["t"].unique())]) + [
    f"Str_{age}" for age in sorted(df_train["t"].unique())
]

df_flat.reset_index(drop=True)

C:\Users\User\AppData\Local\Temp\ipykernel_22416\3301022168.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_flat = data_grouped.apply(


,index,C,BFS,FA,W,SP,Ag,Af,Str_3,Str_7,Str_14,Str_28,Str_56,Str_90,Str_120,Str_180,Str_270,Str_365
0,0,102.0,153.0,0.0,192.0,0.0,887.0,942.0,NaN,7.68,NaN,17.28,NaN,NaN,NaN,NaN,NaN,NaN
1,1,108.3,162.4,0.0,203.5,0.0,938.2,849.0,2.33,NaN,NaN,20.59,NaN,NaN,NaN,NaN,NaN,NaN
2,2,116.0,173.0,0.0,192.0,0.0,909.8,891.9,6.28,10.09,NaN,22.35,NaN,31.02,NaN,NaN,NaN,NaN
3,3,122.6,183.9,0.0,203.5,0.0,958.2,800.1,NaN,10.35,NaN,24.29,NaN,33.19,NaN,NaN,NaN,NaN
4,4,132.0,206.5,160.9,178.9,5.5,866.9,735.6,NaN,NaN,NaN,33.31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,358,528.0,0.0,0.0,185.0,6.9,920.0,720.0,NaN,NaN,NaN,56.83,NaN,NaN,NaN,NaN,NaN,NaN
359,359,531.3,0.0,0.0,141.8,28.2,852.1,893.7,41.30,46.90,NaN,56.40,58.8,59.20,NaN,NaN,NaN,NaN
360,360,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,NaN,NaN,NaN,79.99,NaN,NaN,NaN,NaN,NaN,NaN
361,361,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,NaN,NaN,NaN,61.89,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df_flat[df_flat.Str_28.isnull()].head()

,C,BFS,FA,W,SP,Ag,Af,Str_3,Str_7,Str_14,Str_28,Str_56,Str_90,Str_120,Str_180,Str_270,Str_365
11,135.7,203.5,0.0,185.7,0.0,1076.2,759.3,NaN,7.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,139.6,209.4,0.0,192.0,0.0,1047.0,806.9,8.06,14.59,NaN,NaN,NaN,39.36,NaN,44.21,NaN,44.7
52,153.0,102.0,0.0,192.0,0.0,888.0,943.1,NaN,8.37,NaN,NaN,NaN,26.32,NaN,NaN,NaN,NaN
69,157.0,236.0,0.0,192.0,0.0,935.4,781.2,9.69,NaN,NaN,NaN,NaN,43.38,NaN,NaN,NaN,NaN
87,165.0,128.5,132.1,175.1,8.1,1005.8,746.6,NaN,NaN,NaN,NaN,53.72,NaN,55.02,NaN,NaN,NaN


In [11]:
target_cols = [col for col in df_flat.columns if 'Strength' in col]
df_flat.drop(columns=['index'], inplace=True)


In [12]:
df_flat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   C        363 non-null    float64
 1   BFS      363 non-null    float64
 2   FA       363 non-null    float64
 3   W        363 non-null    float64
 4   SP       363 non-null    float64
 5   Ag       363 non-null    float64
 6   Af       363 non-null    float64
 7   Str_3    94 non-null     float64
 8   Str_7    97 non-null     float64
 9   Str_14   49 non-null     float64
 10  Str_28   312 non-null    float64
 11  Str_56   69 non-null     float64
 12  Str_90   63 non-null     float64
 13  Str_120  42 non-null     float64
 14  Str_180  19 non-null     float64
 15  Str_270  11 non-null     float64
 16  Str_365  16 non-null     float64
dtypes: float64(17)
memory usage: 48.3 KB


In [13]:
target_cols = [col for col in df_flat.columns if 'Str' in col]
X_fl = df_flat.drop(columns=target_cols)
y_fl = df_flat[target_cols]
# y_fl.fillna(0, inplace=True)

In [14]:
y_fl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Str_3    94 non-null     float64
 1   Str_7    97 non-null     float64
 2   Str_14   49 non-null     float64
 3   Str_28   312 non-null    float64
 4   Str_56   69 non-null     float64
 5   Str_90   63 non-null     float64
 6   Str_120  42 non-null     float64
 7   Str_180  19 non-null     float64
 8   Str_270  11 non-null     float64
 9   Str_365  16 non-null     float64
dtypes: float64(10)
memory usage: 28.5 KB


In [1]:
y_fl.head(30)

NameError: name 'y_fl' is not defined

In [14]:
# cb_fl = CatBoostRegressor(
#     iterations=1000,
#     learning_rate=.01,
#     depth=5,
#     random_seed=137,
#     eval_metric='MultiRMSE',
#     logging_level='Silent',
#     loss_function='MultiRMSE',
# )
# funcs.get_cv_rmse(cb_fl, X_fl, y_fl)

Got unsafe target value = nan at object #0 of dataset learn
Got unsafe target value = nan at object #0 of dataset learn
Got unsafe target value = nan at object #0 of dataset learn
Got unsafe target value = nan at object #14 of dataset learn
Got unsafe target value = nan at object #0 of dataset learn
Got unsafe target value = nan at object #0 of dataset learn
Got unsafe target value = nan at object #0 of dataset learn
Got unsafe target value = nan at object #0 of dataset learn
Got unsafe target value = nan at object #0 of dataset learn
Got unsafe target value = nan at object #0 of dataset learn
Got unsafe target value = nan at object #0 of dataset learn
Got unsafe target value = nan at object #1 of dataset learn
Got unsafe target value = nan at object #0 of dataset learn
Got unsafe target value = nan at object #11 of dataset learn
Got unsafe target value = nan at object #0 of dataset learn
Got unsafe target value = nan at object #0 of dataset learn
Got unsafe target value = nan at objec

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\User\skillbox\HOMEWORKS\HWs\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\User\skillbox\HOMEWORKS\HWs\Lib\site-packages\catboost\core.py", line 5873, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, graph, sample_weight, None, None, None, None, baseline,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\skillbox\HOMEWORKS\HWs\Lib\site-packages\catboost\core.py", line 2410

In [18]:
sorted(df_test.t.unique())

[1, 3, 7, 14, 28, 56, 90, 91, 100, 180, 270, 360, 365]

In [ ]:
# report = ProfileReport(df_train)
# report.to_notebook_iframe()

In [ ]:
# from sklearn.neighbors import LocalOutlierFactor

# lof = LocalOutlierFactor(n_neighbors=20)
# outliers = lof.fit_predict(df_train[['Age']])
# filtered_df = df_train[outliers == 1]
# filtered_df.shape

# X = filtered_df.drop(columns=["target"])
# y = filtered_df["target"]

In [ ]:
# df_train["Age_Category"] = pd.cut(
#     df_train["t"], bins=[0, 7, 28, 90, 180, 365], labels=[1, 2, 3, 4, 5]
# ).astype(int)

In [ ]:
# funcs.show_box_plot(df_train)

In [36]:
X = df_train.drop(columns=["target"])
y = df_train["target"]

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=137
# )

In [48]:
cb = CatBoostRegressor(
    iterations=10000,
    learning_rate=.01,
    # cat_features=['t'],
    depth=5,
    random_seed=137,
    eval_metric='RMSE',
    logging_level='Silent',
    loss_function='RMSE',
)
funcs.get_cv_rmse(cb, X, y)

CV RMSE: 4.487866246300864


In [ ]:
cb = CatBoostRegressor(
    iterations=10000,
    learning_rate=.01,
    depth=5,
    random_seed=137,
    eval_metric='RMSE',
    logging_level='Silent',
)
# funcs.get_cv_rmse(cb, X_log, y)

xgb = XGBRegressor(
    random_state=137,
    max_depth=5,
    learning_rate=.01,
    n_estimators=5000
)
# funcs.get_cv_rmse(xgb, X_log, y)

lgb = LGBMRegressor(
    random_state=137,
    verbose=-1,
    n_estimators=5500,
    learning_rate=0.01,
    max_depth=12
)
# funcs.get_cv_rmse(lgb, X_log, y)

rf = RandomForestRegressor(
    random_state=137,
    n_jobs=-1,
    n_estimators=5500,
    max_features="log2",
)
# funcs.get_cv_rmse(rf, X_log, y)

xt = ExtraTreesRegressor(
    random_state=137,
    n_jobs=-1,
    n_estimators=5500,
)
# funcs.get_cv_rmse(xt, X_log, y)

vote = VotingRegressor(
    estimators=[
        ('xt', xt),
        ('rf', rf),
        ('xgb', xgb),
        ('cb', cb),
        ('lgb', lgb),
    ],
    n_jobs=-1,
)

funcs.get_rmse_score([vote], X, y)

RMSE score for VotingRegressor: 4.38


In [ ]:
# vote.fit(X, y)
# y_pred = vote.predict(df_test)
# y_pred_df = pd.DataFrame({'Id': range(0, len(y_pred)), 'Strength': np.round(y_pred, 1)})
# y_pred_df.to_csv('y_pred_regression.csv', index=False)

In [54]:
# df_exp = df_train.copy()
# print(len(df_exp))
# df_exp.drop(columns=["target", "t"], inplace=True)
# duplicates = df_exp[df_exp.duplicated()]
# len(duplicates)


800


0

0

In [69]:
df_test_exp = df_test.copy()

df_test_exp.drop(columns=["t", "Id"], inplace=True)
duplicates_test = df_test_exp[df_test_exp.duplicated()]
len(duplicates_test)


58

In [ ]:
# X_fe = funcs.engineer_features(X)
# # X_test_fe = funcs.engineer_features(X_test_log)

# cb_fe = CatBoostRegressor(
#     iterations=10000,
#     learning_rate=.01,
#     depth=5,
#     random_seed=137,
#     eval_metric='RMSE',
#     logging_level='Silent',
# )
# funcs.get_cv_rmse(cb_fe, X_fe, y)

CV RMSE   : 4.44
MAPE      : 0.12


In [ ]:

cb_fe = CatBoostRegressor(
    iterations=400,
    learning_rate=.5,
    depth=4,
    random_seed=137,
    eval_metric='RMSE',
    logging_level='Silent',
)

xgb_fe = XGBRegressor(
    random_state=137,
    max_depth=5,
    learning_rate=.09,
    n_estimators=500
)

lgb_fe = LGBMRegressor(
    random_state=137,
    verbose=-1,
    n_estimators=550,
    learning_rate=0.08,
    max_depth=12
)

rf_fe = RandomForestRegressor(
    random_state=137,
    n_jobs=-1,
    n_estimators=1500,
    max_features="log2",
)

xt_fe = ExtraTreesRegressor(
    random_state=137,
    n_jobs=-1,
)

vote_fe = VotingRegressor(
    estimators=[
        ('xt_fe', xt_fe),
        ('rf_fe', rf_fe),
        ('xgb_fe', xgb_fe),
        ('cb_fe', cb_fe),
        ('lgb_fe', lgb_fe),
    ],
    n_jobs=-1,
)


funcs.get_rmse_score([vote_fe], X_fe, y)

RMSE score for VotingRegressor: 4.54
